In [ ]:
'''
from google.colab import drive
drive.mount("/content/drive")
'''

Mounted at /content/drive


In [ ]:
'''
from zipfile import ZipFile
file_name="/content/drive/My Drive/Python file/Facial Feature Extraction For Emotion Detection/data.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall("/content/drive/My Drive/Python file/Facial Feature Extraction For Emotion Detection")
  print('Done')
'''

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
import os

In [ ]:
num_classes = 5
img_rows,img_cols = 48,48
batch_size =16


train_data_dir = "/content/drive/My Drive/Python file/Facial Feature Extraction For Emotion Detection/data/train"
validation_data_dir = "/content/drive/My Drive/Python file/Facial Feature Extraction For Emotion Detection/data/validation"

In [ ]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=30,
                    shear_range=0.3,
                    zoom_range=0.3,
                    width_shift_range=0.4,
                    height_shift_range=0.4,
                    horizontal_flip=True,
                    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator = train_datagen.flow_from_directory(
                    train_data_dir,
                    color_mode='grayscale',
                    target_size=(img_rows,img_cols),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                    validation_data_dir,
                    color_mode='grayscale',
                    target_size=(img_rows,img_cols),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True)

Found 24282 images belonging to 5 classes.
Found 5937 images belonging to 5 classes.


In [ ]:
model = Sequential()

# Block-1

model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-2 

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-3

model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-4 

model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-5

model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-6

model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-7

model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
activation_11 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_12 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 24, 24, 32)       

In [ ]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint('/content/drive/My Drive/Python file/Facial Feature Extraction For Emotion Detection/Emotion_2.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [checkpoint,earlystop,reduce_lr]


model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.0001),
              metrics=['accuracy'])

nb_train_samples = 24282
nb_validation_samples = 5937
num_classes = 5
img_rows,img_cols = 48,48
batch_size =16
epochs=30

history=model.fit_generator(
                train_generator,
                steps_per_epoch=nb_train_samples//batch_size,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples//batch_size)

Epoch 1/30
1517/1517 [==============================] - ETA: 0s - loss: 2.2819 - accuracy: 0.2157
Epoch 00001: val_loss improved from inf to 1.59409, saving model to /content/drive/My Drive/Python file/Facial Feature Extraction For Emotion Detection/Emotion_little_vgg_3.h5
1517/1517 [==============================] - 5397s 4s/step - loss: 2.2819 - accuracy: 0.2157 - val_loss: 1.5941 - val_accuracy: 0.2834
Epoch 2/30
1517/1517 [==============================] - ETA: 0s - loss: 1.9602 - accuracy: 0.2311
Epoch 00002: val_loss improved from 1.59409 to 1.54196, saving model to /content/drive/My Drive/Python file/Facial Feature Extraction For Emotion Detection/Emotion_little_vgg_3.h5
1517/1517 [==============================] - 774s 510ms/step - loss: 1.9602 - accuracy: 0.2311 - val_loss: 1.5420 - val_accuracy: 0.3093
Epoch 3/30
1517/1517 [==============================] - ETA: 0s - loss: 1.8017 - accuracy: 0.2401
Epoch 00003: val_loss improved from 1.54196 to 1.53882, saving model to /conte